In [1]:
# %%script echo skipping
import os
from PIL import Image

# Set the directory containing chord images
data_dir = "output"

chords = []
# Collect all chord images into a dictionary
training_data = {}
for filename in os.listdir(data_dir):
    if not filename.endswith(".png"):
        continue
    parts = filename.split("_")
    chord = parts[-2]
    chords.append(chord)
    img_path = os.path.join(data_dir, filename)
    training_data[img_path] = chord

In [2]:
chord_dict = {}
for chord in list(training_data.values()):
    if chord not in chord_dict:
        chord_dict[chord] = len(chord_dict)

In [3]:
import re

chords_data = {}
unique_chords = [*set(chord_dict)]

unique_chords.append('Am7/G')
unique_chords.append('CMaj67/E')
unique_chords.append('G7alt')  
unique_chords.append('C#aug') 
unique_chords.append('E♭aug/F') 


def checkIfDominant(string):
    return string[1] == '7' or string[1] == '9' or string[1:3] == '13' or string[1:3] == '67'
    
def getDominantType(string):
        if string[1:3] == '13':
            return "13"
        if string[1:3] == '11':
            return "11"
        if string[1] == '9':
            return "9"
        if string[1] == '7':
            return "7"
        # if string[1:3] == '67':
        #     return 'add6'

In [4]:
def getChordType(chord_noAcc, major, minor, diminished, dominant):
    
    if minor or diminished:
        if chord_noAcc[2] == "7":
            return "min7"
        # if chord_noAcc[2] == "6":
        #     return "min6"
        elif chord_noAcc[2] == "9" or chord_noAcc[2] == "6": 
            return 'min' + chord_noAcc[2]
        elif chord_noAcc[2] == "1": 
            return 'min' + chord_noAcc[2:4]
        else:
            return "min5"
            
    if major:
        if chord_noAcc[4:6] == "67":
            return "maj67"
        if chord_noAcc[4] == "7":
            return "maj7"
        if chord_noAcc[4] == "9": 
            return 'maj9'
        # if chord_noAcc[4] == "6": ## Notation like C6 is used without maj.
            # return "maj6"
        elif chord_noAcc[4] == "1": 
            return 'maj' + chord_noAcc[4:6]
            
    if dominant:
        return 'dominant' + dominant_type
        
    if not dominant and not major and not minor and not diminished:
        try:
            if chord_noAcc[1:4] == "aug":
                return "aug"
            if chord_noAcc[1:3] == "69":
                return "69"
            elif chord_noAcc[1] == "6":
                return "6"
            # if chord_noAcc[1:3] == "":
            #     return ""
            else:
                return "maj5"
                # return "uncertain"
        except IndexError:
            print('ERROR!')

In [5]:
def getAddDegree(chord):
    add_accidental = "N/A"
    add_idx = None
    addNote_degree = "N/A"

    add_idx = chord.find('add')

    if chord[add_idx+3] == '♭' or chord[add_idx+3] == '#':    ## Accidental found in add!
        add_accidental = chord[add_idx+3]

    if add_accidental == "N/A":
        if chord[add_idx+3] == "9":
            addNote_degree = "9"
        if chord[add_idx+3] == "6":
            addNote_degree = "6"
        if chord[add_idx+3] == "4":
            addNote_degree = "4"
        elif chord[add_idx+3] == "1":
            addNote_degree = chord[add_idx+3:chord[add_idx+4]]

    if add_accidental != "N/A":
        chord_addAccRemoved = chord[:add_idx+3] + chord[add_idx+4:]
        if chord_addAccRemoved[add_idx+3] == "9":
            addNote_degree =  "9"
        if chord_addAccRemoved[add_idx+3] == "6":
            addNote_degree = "6"
        elif chord_addAccRemoved[add_idx+3] == "1":
            addNote_degree = chord_addAccRemoved[add_idx+3:add_idx+5]
        
    return addNote_degree, add_accidental

In [6]:
for chord in unique_chords:

    root = chord[0]
    root_accidental = "♮"
    chord_noAcc = chord
    
    if chord[1] == "♭" or chord[1] == "#":
        root_accidental = chord[1]
        chord_noAcc = chord_noAcc[0:1] + chord_noAcc[2:]     ## Remove accidental for easier parsing
    
    dominant = checkIfDominant(chord_noAcc)
    dominant_type = "N/A" 
    
    if dominant:
        dominant_type = getDominantType(chord_noAcc)
            
    diminished = chord_noAcc[1] == 'o'
    minor = chord_noAcc[1] == 'm'
    
    major = 'Maj' in chord_noAcc[1:4] 
    sus4 = 'sus4' in chord_noAcc
    
    over_note = ''
    alteration = []
    matches = []
    
    ## Check for '(♭5)' in  Fm9(♭5)
    if '/' in chord:
        index = chord.find('/')
        over_note = chord[index+1:]
        
    if '(' in chord:
        start_index = chord.find('(')
        end_index = chord.find(')')
        
        alteration = chord[start_index+1:end_index]       
        pattern = "([♭#]\d+)"
        matches =  re.findall(pattern, alteration)        
      
    add_present = 'add' in chord or 'Δ' in chord
    add_accidental = "N/A"
    addNote_degree = "N/A"
    
    if add_present:
        add_idx = chord.find('add')
        addNote_degree, add_accidental = getAddDegree(chord)

    chord_type = ""
    chord_type = getChordType(chord_noAcc, major, minor, diminished, dominant)


    chords_data[chord] = {
            'root': root,
            'root_accidental': root_accidental,
            'chord_type': chord_type,
            'slash_note': over_note,
            'alternations': matches,
            'diminished_substring': diminished,
            'major_substring': major,
            'minor_substring': minor,
            'dominant': dominant,
            'dominant_type': dominant_type,
            'sus4_substring': sus4,
            'addNote_accidental': add_accidental,
            'addNote_degree': addNote_degree,
        }

In [7]:
# for chord in chords_list:
#     if chord == "Am7/G":
#         print(key)
    # for keyInner, valueInner in value.items():
        # if keyInner == "chord_type":
        # if keyInner == "chord_type":
            # print(key)
            # if valueInner == "uncertain":
            # # if valueInner == "min5th":
            #     # min5th
            #     print(key, value['chord_type'])
            #     print(value)

In [8]:
for key, value in chords_data.items():
    # if key == 'Am7/G':
    #     print(key, value)
    for keyInner, valueInner in value.items():
        if keyInner == "chord_type":
            # print(key)
            if valueInner == "uncertain":
            # if valueInner == "min5th":
                # min5th
                print(key, value['chord_type'])
                # print(value)
# print('\n\n',chords_data)
# my_dict = chords_data
# print(list(chords_data.keys()))
# print([s for s in list(chords_data.keys()) if '6' in s])

In [9]:
%%script echo skipping

chords_list = list(chords_data.keys())
print('----------- contains 6 -----------')
for c in chords_list:
    if '6' in c:
        print(c, chords_data[c], '\n')
        
print('\n \n')

print('----------- chord_type = maj5 -----------')
for c in chords_list:
    # print(chords_data[c])
    if chords_data[c]['chord_type'] == 'maj5':
        print(c, chords_data[c], '\n')
        
print('\n \n')
        
print('----------- not Major Minor Dom Diminished -----------')
for c in chords_list:
    # print(chords_data[c])
    if not chords_data[c]['major_substring'] and not chords_data[c]['minor_substring'] and not chords_data[c]['dominant'] and not chords_data[c]['diminished_substring']:
        print(c, chords_data[c], '\n')

skipping


In [29]:
unique_keys = {}

for obj in chords_data:
    for key in chords_data[obj]:
        if key not in unique_keys:
            unique_keys[key] = set()

        value = chords_data[obj][key]
        # if key == "alternations":
        #     print(value)
        if isinstance(value, list):
            value = tuple(value)
        unique_keys[key].add(value)
        
        

for key in unique_keys:
    print('\n')
    print(f"Unique values for **{key}**:")
    # print("Unique values for key", key, ":")
    print('--------------------------------------')
    for value in unique_keys[key]:
        if type(value) == tuple:
            print(list(value))
        else:
            print(f'"{value}"', ",")
    print('--------------------------------------')
    print('\n')



Unique values for **root**:
--------------------------------------
"D" ,
"G" ,
"F" ,
"A" ,
"B" ,
"C" ,
"E" ,
--------------------------------------




Unique values for **root_accidental**:
--------------------------------------
"#" ,
"♭" ,
"♮" ,
--------------------------------------




Unique values for **chord_type**:
--------------------------------------
"6" ,
"maj9" ,
"maj5" ,
"dominant13" ,
"min11" ,
"min6" ,
"69" ,
"aug" ,
"maj67" ,
"min5" ,
"dominant7" ,
"maj7" ,
"min7" ,
"min9" ,
"dominant9" ,
--------------------------------------




Unique values for **slash_note**:
--------------------------------------
"" ,
"G" ,
"F" ,
"E" ,
--------------------------------------




Unique values for **alternations**:
--------------------------------------
['#9']
['♭6']
['♭5', '#9']
['♭13']
['♭5']
['#5']
['♭9', '#11']
['#5', '#9']
['#11']
[]
['♭9']
--------------------------------------




Unique values for **diminished_substring**:
--------------------------------------
"False" ,


### Note
Csus4(♭9) chord is not a dominant chord.

C9sus4 is a dominant chord.

Also note:

Csus4(♭9) is not considered major, minor, or diminished because it does not have a major or minor third. The third is replaced with a suspended fourth, and the chord contains a flat ninth (♭9) instead.


// Modification Types.

    // Sus4 replaces 3 with 4.

    // Six-Chords add 6th.
    
-----
    
Cadd9 and C(Δ9) are essentially the same chord.

----
"Ao7add♭13" and "Ao add♭13"

The main difference between the two chords is that the "Ao7add♭13" chord contains a diminished seventh, while the "Ao add♭13" chord does not. This can affect the way the chord sounds and functions in a musical context.
-----

"Cm7add4" and "Cm47"